# Topic modeling

Zadanie proszę wykonać w parach, ale... na dwóch komputerach. Na obu komputerach proszę najpierw wykonać instrukcje z sekcji **Przygotowanie**. Następnie na jednym komputerze proszę od razu odkomentować i uruchomić kod w sekcji **Wikipedia** i **Wizualizacja**. Na drugim komputerze proszę rozwiązać zadania z sekcji **Tweety** i **Wizualizacja**.

Po wykonaniu tego zadania powinieneś:
+ potrafić wykonać podstawowy topic modeling,
+ umieć stworzyć słownik mapujący identyfikatory na słowa,
+ potrafić stworzyć macierz wektorów TF-IDF,
+ wiedzieć jak wykorzystać LDA do określenia proporcji tematów w nowym dokumencie tekstowym,
+ potrafić zwizualizować wyniki algorytmu LDA.

Do wykonania zadania wykorzystamy bibliotekę [gensim](https://radimrehurek.com/gensim/), która oferuje szereg metod do analizy tekstu. Warto w wolnej chwili zobaczyć co oprócz algorytmu LDA zostało zaimplementowane w ramach tego modułu!

## Przygotowanie

Aby zadziałała wizualizacja, musimy najpierw zauktualizować bibliotekę `scipy` i doinstalować bibliotekę `pyldavis`. Będzie to okazja, żeby zobaczyć jak zarządza się bibliotekami w anacondzie. Jeśli kogoś interesuje co oznaczają kolejne komendy, proszę zajrzeć do [dokumentacji Anacondy](http://conda.pydata.org/docs/using/pkgs.html).

1. Zatrzymaj kernel (serwer jupyter notebooka)
2. Otwórz terminal
3. Wpisz `conda update scipy` (i Enter gdy spytają `Proceed`)
4. Popatrz na paski postępu
5. Wpisz `activate root`
6. Wpisz `pip install pyldavis`
7. Wpisz `deactivate`

Koniec. Możesz ponownie odpalić notatnik i przejść do kolejnych kroków.

## Wikipedia

Ten fragemnt kodu stanowi przykład uruchomienia topic modelingu na większym zbiorze danych. Ponieważ wyliczenie modelu będzie trwać od kilku do kilkunastu minut, niech każda para uruchomi ten przykład tylko na jednym komputerze.

Aby uruchomić przykład, w folderze z notatnikiem muszą znajdować się pliki `wiki_wordids.txt.bz2` i `wiki_tfidf.mm` ściągnięte wraz z notatnikiem. Przykład zbudowany jest w oparciu o podzbiór stron wikipedii dostępny pod adresem: https://dumps.wikimedia.org/enwiki/latest/. Strony zostały przekonwertowane na reprezentację wektorową za pomocą skryptu:
`python -m gensim.scripts.make_wiki`.

**Przeczytaj komentarze zanim uruchomisz kod.**

In [ ]:
# import logging
# import gensim

# # Włączamy logowanie, żeby śledzić postępy algorytmu (to akurat nie będzie działać w Jupyter Notebooku,
# # ale warto o tym wspomnieć)
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# # Odczytujemy z pliku mapowanie/słownik id->słowo
# id2word = gensim.corpora.Dictionary.load_from_text('wiki_wordids.txt.bz2')

# # Odczytujemy z pliku reprezentację wektorową korpusu (macierz wetkorów TF-IDF)
# mm = gensim.corpora.MmCorpus('wiki_tfidf.mm')
# print(mm)

# # Tworzymy model LDA z 20 grupami wykonując 20 iteracji na całym zbiorze
# lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=20, passes=20, workers=4)
# # Alternatywnie w razie problemów z wielowątkowością:
# # lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=20, update_every=0, passes=20)
# lda.print_topics(20)

## Tweety

W parach stwórzcie model LDA w oparciu o załączone Tweety. W tym celu należy przekonwertować pliki tekstowe na reprezentację wektorową zapisując wcześniej mapowanie id->słowo w postaci słownika. Opis jak stworzyć wymienione struktury danych można znaleźć na stronie: https://radimrehurek.com/gensim/tut1.html.

**Zad. 1: Wczytaj tweety z pliku tweets.tsv do zmiennej `tweets`.**

In [ ]:
import gensim
import logging
import nltk
import re
import pandas as pd

stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
            "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've",
            "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "of",
            "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "shan't", "she", "she'd",
            "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs",
            "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're",
            "they've", "this", "those", "through", "to", "until", "up", "very", "was", "wasn't", "we", "we'd",
            "we'll", "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's",
            "which", "while", "who", "who's", "whom", "with", "would", "you", "you'd", "you'll", "you're", "you've",
            "your", "yours", "yourself", "yourselves", "above", "again", "against", "aren't", "below", "but", "can't",
            "cannot", "couldn't", "didn't", "doesn't", "don't", "down", "few", "hadn't", "hasn't", "haven't", "if",
            "isn't", "mustn't", "no", "nor", "not", "off", "out", "over", "shouldn't", "same", "too", "under", "why",
            "why's", "won't", "wouldn't"]

tweets = None

**Zad. 2: Dokonaj tokenizacji słów, usuń te z stoplisty (`stopwords`) oraz występujące tylko raz. Wynik przypisz do zmiennej `texts`.**

In [ ]:
texts = None 

**Zad. 3: Stwórz słownik id->słowo i przypisz do zmiennej `id2word`.**

In [ ]:
id2word = None

**Zad. 4: Stwórz reprezentację wektorową korpusu (macierz wektorów TF-IDF), wynik przypisz do zmiennej `mm`.**

In [ ]:
mm=None

**Zad. 5: Odpal poniższy kod i odkryj 10 tematów za pomocą algorytmu LDA. Jeśli masz czas, zwiększ wartości parametrów num_topics i passes przy tworzeniu modelu LDA, i sprawdź jak to wpłynie na rezultat.**

In [ ]:
# # Włączamy logowanie, żeby śledzić postępy algorytmu (to akurat nie będzie działać w Jupyter Notebooku,
# # ale warto o tym wspomnieć)
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#
# lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=10, update_every=0, passes=20)
# # alternatywnie lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=10, passes=20)
# lda.print_topics(10)

**Zad. 6*: Na podstawie zbudowanego modelu określ proporcje tematów w następującym tweecie:
`Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season`. Jeśli zostało niewiele czasu, przejdź od razu do wizualizacji.**

In [ ]:
# new_tweet = "Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season"
# new_tweet_text = ???
# new_tweet_mm = id2word.doc2bow(new_tweet_text)
# new_tweet_lda = ???
# print(new_tweet_lda)

## Wizualizacja

Spróbujemy teraz zwizualizować uzyskane tematy. Podążaj za komentarzami a powinno się udać.

P.S. Możesz uruchomić tę wizualizację również dla tematów odkrytych z Wikipedii...

**Zad. 7: Zwizualizuj uzyskane tematy.**

In [ ]:
# # Poniższy kod korzysta modułu pyLDAvis z githuba:
# # https://github.com/bmabey/pyLDAvis

# import pyLDAvis
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()

# pyLDAvis.gensim.prepare(lda, mm, id2word)